In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
%matplotlin inline

UsageError: Line magic function `%matplotlin` not found.


In [2]:
emp_df=pd.read_csv('employee.csv')

In [3]:
employee=emp_df.drop(['EmployeeCount','EmployeeNumber','Over18','StandardHours'],axis=1)

In [4]:
employee['Attrition'].replace(['Yes','No'],[1,0],inplace=True)

In [5]:
numerics = ['int64']

employee_num = employee.select_dtypes(include=numerics)

In [6]:
char = ['object']

employee_char = employee.select_dtypes(include=char)

In [7]:
employee_dummies=pd.get_dummies(employee_char)

In [8]:
employee_combined=pd.concat([employee_num,employee_dummies],axis=1)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
train_x=employee_combined.drop(['Attrition'],axis=1)

In [11]:
train_y=employee_combined['Attrition']

In [12]:
X_train,X_test,y_train,y_test=train_test_split(train_x,train_y,test_size=0.3,random_state=20)

In [16]:
y_test.shape

(441,)

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
rf=RandomForestClassifier(n_estimators=100)

In [20]:
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [21]:
y_pred=rf.predict(X_test)

In [97]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print('Accuracy :', accuracy)
print('precision :', precision)
print('recall :', recall)

Accuracy : 0.8639455782312925
precision : 0.6666666666666666
recall : 0.2857142857142857


In [39]:
#Tuning tree using random search
from time import time
from scipy.stats import randint as sp_randint
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier



In [61]:
#utility method for reporting parameters
def report(results,ntop=3):
    for i in range(1,ntop+1):
        candidates=np.flatnonzero(results['rank_test_score']==i)
        for candidate in candidates:
            print("Model with Rank: {0}".format(i))
            print("model validation score: {0:.3f} (std:{1:.3f})".format(results['mean_test_score'][candidate],results['std_test_score'][candidate]))
            print("Parameters: {0}",format(results['params'][candidate]))
            print("")
        

In [62]:
#specifying parameters and distribution to sample form
param_dist={'max_depth':[3,None],
           'max_features':sp_randint(1,11),
           'min_samples_split':sp_randint(2,11),
           'bootstrap':[True,False],
           'criterion':['gini','entropy']}

In [63]:
#run randamized search
n_iter_search=20
random_search=RandomizedSearchCV(rf,param_distributions=param_dist,n_iter=n_iter_search,cv=5)

In [64]:
start=time()
random_search.fit(X_train,y_train)
print("RandomizedSearchCv took %.2f sec for %d candidates"
      "Parameter setting."%((time()-start),n_iter_search))
report(random_search.cv_results_)

RandomizedSearchCv took 19.15 sec for 20 candidatesParameter setting.
Model with Rank: 1
model validation score: 0.866 (std:0.011)
Parameters: {0} {'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'max_features': 8, 'min_samples_split': 7}

Model with Rank: 2
model validation score: 0.864 (std:0.013)
Parameters: {0} {'bootstrap': False, 'criterion': 'gini', 'max_depth': None, 'max_features': 7, 'min_samples_split': 6}

Model with Rank: 3
model validation score: 0.863 (std:0.014)
Parameters: {0} {'bootstrap': True, 'criterion': 'entropy', 'max_depth': None, 'max_features': 10, 'min_samples_split': 2}



In [65]:
random_search.cv_results_

C:\Users\visha\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\visha\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\visha\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\visha\Anaconda3\lib\site-packages\sklearn\util

{'mean_fit_time': array([0.09135818, 0.12266474, 0.14980812, 0.23836308, 0.19328256,
        0.14880128, 0.1623652 , 0.17234464, 0.16576467, 0.09713988,
        0.2152257 , 0.13284349, 0.13563809, 0.15678024, 0.10231805,
        0.18850336, 0.10212703, 0.28881989, 0.08756566, 0.22539673]),
 'std_fit_time': array([0.00868313, 0.00259451, 0.00074167, 0.0049663 , 0.00420272,
        0.00146595, 0.02359507, 0.00602709, 0.0016102 , 0.00048926,
        0.00494253, 0.00074846, 0.00126173, 0.00212988, 0.0004827 ,
        0.00303427, 0.00048992, 0.00349684, 0.00039911, 0.00488566]),
 'mean_score_time': array([0.00918326, 0.00917401, 0.0107707 , 0.01077423, 0.01117082,
        0.01176887, 0.01156869, 0.01117125, 0.0109694 , 0.00897579,
        0.01096926, 0.00937629, 0.0091753 , 0.01077285, 0.00957427,
        0.01176848, 0.00957518, 0.01116285, 0.00957422, 0.01117043]),
 'std_score_time': array([3.96698385e-04, 3.99880758e-04, 4.03123210e-04, 3.99765600e-04,
        3.98898677e-04, 3.99017419e-

(1029, 51)

In [66]:
#grid search
#use a full grid over all parameters
param_grid={'max_depth':[3,None],
           'max_features':[1,3,10],
           'min_samples_split':[2,3,10],
           'bootstrap':[True,False],
           'criterion':['gini','entropy']}


In [67]:
#run grid search
grid_search=GridSearchCV(rf,param_grid=param_grid,cv=5)

In [69]:
start=time()
grid_search.fit(X_train,y_train)
print('GridSearchCV took %.2f sec for %d candidates parameter setting'
     % ((time()-start,len(grid_search.cv_results_['params']))))
report(grid_search.cv_results_)

GridSearchCV took 65.53 sec for 72 candidates parameter setting
Model with Rank: 1
model validation score: 0.869 (std:0.011)
Parameters: {0} {'bootstrap': False, 'criterion': 'gini', 'max_depth': None, 'max_features': 10, 'min_samples_split': 10}

Model with Rank: 2
model validation score: 0.867 (std:0.012)
Parameters: {0} {'bootstrap': False, 'criterion': 'gini', 'max_depth': None, 'max_features': 10, 'min_samples_split': 2}

Model with Rank: 3
model validation score: 0.866 (std:0.012)
Parameters: {0} {'bootstrap': False, 'criterion': 'gini', 'max_depth': None, 'max_features': 10, 'min_samples_split': 3}

Model with Rank: 3
model validation score: 0.866 (std:0.012)
Parameters: {0} {'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'max_features': 10, 'min_samples_split': 2}



In [70]:
grid_search.cv_results_

C:\Users\visha\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\visha\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\visha\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\visha\Anaconda3\lib\site-packages\sklearn\util

{'mean_fit_time': array([0.12179375, 0.08978796, 0.09372745, 0.09371753, 0.09373446,
        0.09372582, 0.12183914, 0.12275543, 0.16559358, 0.17495937,
        0.12184525, 0.12497778, 0.14058485, 0.13121953, 0.12496157,
        0.21244922, 0.20620227, 0.24368196, 0.09060211, 0.08436236,
        0.09372721, 0.09684405, 0.0968493 , 0.09685025, 0.13746414,
        0.13741179, 0.12496819, 0.1593235 , 0.13435125, 0.11248102,
        0.19370365, 0.14996471, 0.14369774, 0.24055867, 0.23744173,
        0.23743911, 0.08747902, 0.08122673, 0.08435159, 0.09371724,
        0.0937356 , 0.0937366 , 0.16558418, 0.13433409, 0.13433442,
        0.14995542, 0.16245708, 0.12184629, 0.16246734, 0.15933137,
        0.146838  , 0.28743186, 0.29055414, 0.27806735, 0.08747764,
        0.08748322, 0.08122439, 0.10935278, 0.10624032, 0.10542126,
        0.15621238, 0.15309439, 0.16104712, 0.18121314, 0.1437098 ,
        0.12809286, 0.18433299, 0.18747177, 0.16872377, 0.34367647,
        0.34367814, 0.3218009 ]

In [87]:
#now run the random forest with the best paramaters of search
rf_random_search=RandomForestClassifier(n_estimators=100,bootstrap=True,criterion='gini',max_depth=None,max_features=10,
                                       min_samples_split=10)

In [88]:
rf_random_search.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=10, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [89]:
y_pred=rf_random_search.predict(X_test)

In [95]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [98]:
print('accuracy using random forest random serach is:',accuracy_score(y_test,y_pred))

accuracy using random forest random serach is: 0.8639455782312925


In [99]:
print('precision is',precision_score(y_test,y_pred))

precision is 0.6666666666666666


In [76]:
from sklearn.metrics import accuracy_score

In [100]:
#build classifier using grid serach
rf_grid_search=RandomForestClassifier(n_estimators=100,bootstrap=False,criterion='gini',max_features=10,min_samples_split=2)

In [101]:
rf_grid_search.fit(X_train,y_train)

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features=10, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [102]:
y_pred=rf_grid_search.predict(X_test)

In [103]:
print('Accuracy',accuracy_score(y_test,y_pred))

Accuracy 0.8594104308390023


In [104]:
print('precision',precision_score(y_test,y_pred))

precision 0.6538461538461539


In [105]:
print('recall',recall_score(y_test,y_pred))

recall 0.24285714285714285
